In [ ]:
import numpy as np

# Fill out the correct directory in which the values are stored here
# The path must not end with /
exp_dir = None

if exp_dir is None:
    raise Exception("exp_dir must be given")

exp_dir = str(exp_dir)

# Parameters of the training
n_gt = 10
nl_dict = {3: [1, 2, 3, 5], 5: [1, 3, 5, 10], 10: [1, 5, 10, 15]}

mean_losses_lin = np.load("%s/savepoints/mean_losses_lin.npy" % exp_dir, allow_pickle=True).item()
mean_losses_sm = np.load("%s/savepoints/mean_losses_sm.npy" % exp_dir, allow_pickle=True).item()

std_losses_lin = np.load("%s/savepoints/std_losses_lin.npy" % exp_dir, allow_pickle=True).item()
std_losses_sm = np.load("%s/savepoints/std_losses_sm.npy" % exp_dir, allow_pickle=True).item()

losses_lin = np.load("%s/savepoints/losses_lin.npy" % exp_dir, allow_pickle=True).item()
losses_sm = np.load("%s/savepoints/losses_sm.npy" % exp_dir, allow_pickle=True).item()
        

In [ ]:
import matplotlib.pyplot as plt

SMALL_SIZE = 15
MEDIUM_SIZE = 30
BIGGER_SIZE = 35

plt.rc('font', size=BIGGER_SIZE)# controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)
plt.rc('axes', labelsize=BIGGER_SIZE, linewidth=5)     # fontsize of the axes title # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('xtick.major', width=5, size=20)
plt.rc('xtick.minor', width=5, size=20)
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick.major', width=5, size=20)
plt.rc('ytick.minor', width=5, size=20)
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('lines', linewidth=5)


ldivn_scatter_sm = {float(l)/float(n): [] for n in [3, 5, 10] for l in sorted(nl_dict[n])}
for n in [3, 5, 10]:
    for l in sorted(nl_dict[n]):
        ldivn = float(l)/float(n)
        loss_nl_sm, loss_nl_lin = [], []
        for i in range(10):
            ldivn_scatter_sm[ldivn] += losses_sm[(i, n, l)]
            loss_nl_sm.append(losses_sm[(i, n, l)])
            loss_nl_lin.append(losses_lin[(i, n, l)])
        print("%d & %d & %.3f (%.3f/%.3f) & {%.3f (%.3f/%.3f)} \\\\" % (n, l, np.quantile(loss_nl_lin, 0.5), np.quantile(loss_nl_lin, 0.25), 
                                                                        np.quantile(loss_nl_lin, 0.75), np.quantile(loss_nl_sm, 0.5), 
                                                                        np.quantile(loss_nl_sm, 0.25), np.quantile(loss_nl_sm, 0.75)))
        

ldivn_scatter_lin = {float(l)/float(n): [] for n in [3, 5, 10] for l in sorted(nl_dict[n])}
for n in [3, 5, 10]:
    for l in sorted(nl_dict[n]):
        ldivn = float(l)/float(n)
        for i in range(9):
            ldivn_scatter_lin[ldivn] += losses_lin[(i, n, l)]

mean_by_ldivn_sm = []
stdm_by_ldivn_sm = []
stdp_by_ldivn_sm = []
quantile_75_by_ldivn_sm = []
quantile_25_by_ldivn_sm = []
median_by_ldivn_sm = []
ldivns = sorted(ldivn_scatter_sm.keys())
print(ldivns)
for ldivn in ldivns:
    data = ldivn_scatter_sm[ldivn]
    mean = np.mean(data)
    mean_by_ldivn_sm.append(mean)
    stdm_by_ldivn_sm.append(mean - np.std(data))
    stdp_by_ldivn_sm.append(max(0, mean + np.std(data)))
    quantile_75_by_ldivn_sm.append(np.quantile(data, 0.75))
    quantile_25_by_ldivn_sm.append(np.quantile(data, 0.25))
    median_by_ldivn_sm.append(np.quantile(data, 0.5))
    
    
mean_by_ldivn_lin = []
stdm_by_ldivn_lin = []
stdp_by_ldivn_lin = []
quantile_75_by_ldivn_lin = []
quantile_25_by_ldivn_lin = []
median_by_ldivn_lin  = []
ldivns = sorted(ldivn_scatter_lin.keys())
for ldivn in ldivns:
    data = ldivn_scatter_lin[ldivn]
    mean = np.mean(data)
    mean_by_ldivn_lin.append(mean)
    stdm_by_ldivn_lin.append(mean - np.std(data))
    stdp_by_ldivn_lin.append(max(0, mean + np.std(data)))
    quantile_75_by_ldivn_lin.append(np.quantile(data, 0.75))
    quantile_25_by_ldivn_lin.append(np.quantile(data, 0.25))
    median_by_ldivn_lin.append(np.quantile(data, 0.5))

x = range(len(mean_by_ldivn_sm))
plt.plot(x, median_by_ldivn_sm, marker='x', color='green', label='Softmax', alpha=0.7)
for x_val in x:
    plt.plot([x_val, x_val], [quantile_25_by_ldivn_sm[x_val], quantile_75_by_ldivn_sm[x_val]], color='green', linewidth=4, markersize=18, markeredgewidth=3, alpha=0.5, marker='_')

x = range(len(mean_by_ldivn_lin))
plt.plot(x, median_by_ldivn_lin, marker='x', color='blue', label='normAbsLin', alpha=0.7)
for x_val in x:
    plt.plot([x_val, x_val], [quantile_25_by_ldivn_lin[x_val], quantile_75_by_ldivn_lin[x_val]], color='blue', linewidth=4, markersize=18, markeredgewidth=3, alpha=0.5, marker='_')

pos_of_1 = np.where(np.array(ldivns) == 1)[0][0]
plt.plot([pos_of_1, pos_of_1], [-0.19, 1], '--', color='gray')
    
plt.xticks([_ for _ in range(len(ldivns))], ['%.2f' % ldivn for ldivn in ldivns])
plt.ylabel('loss')
plt.xlabel('l/n')
plt.legend()
plt.ylim(-0.19, 1.)
fig = plt.gcf()
fig.set_size_inches(19, 10)
plt.xticks(rotation=45)
file='%s/result.png' % exp_dir
if file is not None:
    plt.savefig(file, bbox_inches='tight')
fig.show()